In [1]:
from pyspark import SparkContext
# $example on$
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.sql import SQLContext, Row
from pyspark.mllib.linalg import Matrix, Matrices
from decimal import *
from pyspark.sql.functions import col
from pyspark.mllib.linalg import DenseVector, DenseMatrix
import numpy as np

# $example off$

sqlContext = SQLContext(sc)
path = 'res_comment.csv'

df = sqlContext.read.csv(path, header=False)
df.show(5)
df.count()

# df.show(5)
df.count()


+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+----------+-----------+----------

23386

In [2]:
FEATURES_COL = df.schema.names[1:]
# df.printSchema()
# FEATURES_COL = df.
df_comment = df.select(*(df[c].cast("float").alias(c) for c in df.columns[1:101]))
df_comment.dropna(how='any')
df_comment.count()
df_comment.show(5)
# rdd_comment = df_comment.rdd
#print(type(rdd_comment))

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+----------+-----------+----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+----------+----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-----------+----------+----------+----------+----------+-----------+----------+-----------+----------+----------+-----------+-----------+-----------+----------+-----------+-----------+-----------+-----------+----------

In [3]:
if __name__ == "__main__":

    mat = RowMatrix(df_comment.rdd.map(lambda v: Vectors.dense(v)))
    
    # Compute the top 20 singular values and corresponding singular vectors.
    n_svd = 20
    svd = mat.computeSVD(n_svd, computeU=True)
    U = svd.U       # The U factor is a RowMatrix.
    s = svd.s       # The singular values are stored in a local dense vector.
    V = svd.V       # The V factor is a local dense matrix.

    collected = U.rows.collect()
    print("U factor is:")
#     for vector in collected:
#         print(vector)

    mat_s = DenseMatrix(len(s), len(s), np.diag(s).ravel("F"))
    
#     t_list = [0.0 for i in range(n_svd**2)]
#     for i in range(n_svd):
#         t_list[i + i*n_svd] = s[i]
    
#   print(t_list)
    print("V factor is:\n%s" % V)
#     mat_s = Matrices.dense(n_svd, n_svd, t_list)
#     print(s_mat)
    mat_us = U.multiply(mat_s)

    V_ = DenseMatrix(V.numCols, V.numRows, V.toArray().transpose().ravel("F"))
#     V_transpose_cols = len(V.toArray())
#     print(V_transpose_cols)
#     V_transpose = DenseMatrix(V_transpose_cols, n_svd, V.toArray(), True)
#     print(type(V))
    mat_usv = mat_us.multiply(V_)
        
    print("Singular values are: %s" % s)
    print("V factor is:\n%s" % V)

U factor is:
V factor is:
DenseMatrix([[-0.251477  ,  0.20721859, -0.41129006, ..., -0.01667235,
              -0.10221111, -0.03469437],
             [ 0.44830842,  0.03745748,  0.30363899, ...,  0.00067182,
               0.07745276, -0.06731139],
             [-0.14416074, -0.31200721,  0.16488212, ...,  0.0189935 ,
               0.02125434,  0.13367514],
             ...,
             [-0.02593501, -0.03941091,  0.01181299, ..., -0.04286221,
              -0.0900616 ,  0.01739824],
             [-0.01376223, -0.02598951, -0.00879889, ..., -0.06720057,
               0.16097534,  0.00578034],
             [ 0.04454415,  0.08009934, -0.01457313, ...,  0.04345032,
               0.03891119,  0.08533275]])
Singular values are: [90.44909640320824,90.00930148679718,86.22474404505047,83.52197257548721,80.54981567971168,78.63004742939671,78.42259236967827,77.92543753287957,77.43301797240247,77.16836291198979,76.76296560209647,76.32557315176545,75.99487467616505,75.60451192338545,75.440142

In [4]:
    rdd_usv = mat_usv.rows.map(
         lambda x: Row(_c1=Decimal(x[0]), _c2=Decimal(x[1]), _c3=Decimal(x[2]),
                       _c4=Decimal(x[3]), _c5=Decimal(x[4]), _c6=Decimal(x[5]),
                       _c7=Decimal(x[6]), _c8=Decimal(x[7]), _c9=Decimal(x[8]),
                       _c10=Decimal(x[9]), _c11=Decimal(x[10]), _c12=Decimal(x[11]),
                       _c13=Decimal(x[12]), _c14=Decimal(x[13]), _c15=Decimal(x[14]),
                       _c16=Decimal(x[15]), _c17=Decimal(x[16]), _c18=Decimal(x[17]),
                       _c19=Decimal(x[18]), _c20=Decimal(x[19])))
#                                _c22=Decimal(x[21]), _c23=Decimal(x[22]), _c24=Decimal(x[23]),
#                                _c25=Decimal(x[24]), _c26=Decimal(x[25]), _c27=Decimal(x[26]),
#                                _c28=Decimal(x[27]), _c29=Decimal(x[28]), _c30=Decimal(x[29]),
#                                _c31=Decimal(x[30]), _c32=Decimal(x[31]), _c33=Decimal(x[32]),
#                                _c34=Decimal(x[33]), _c35=Decimal(x[34]), _c36=Decimal(x[35]),
#                                _c37=Decimal(x[36]), _c38=Decimal(x[37]), _c39=Decimal(x[38]),
#                                _c40=Decimal(x[39]), _c41=Decimal(x[40]), _c42=Decimal(x[41]),
#                                _c43=Decimal(x[42]), _c44=Decimal(x[43]), _c45=Decimal(x[44]),
#                                _c46=Decimal(x[45]), _c47=Decimal(x[46]), _c48=Decimal(x[47]),
#                                _c49=Decimal(x[48]), _c50=Decimal(x[49])))

    df_usv = rdd_usv.toDF()
    df_usv.fillna(0.0)
    df_usv.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _c1|                _c10|                _c11|                _c12|                _c13|                _c14|                _c15|                _c16|                _c17|                _c18|                _c19|                 _c2|                _c20|                 _c3|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|                 _c9|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [5]:
# df_us.withColumn("_c0", df["_c0"])
print(df.count())
print(df_usv.count())

23386
23386


In [57]:
# from pyspark.sql.functions import col
# def addColumnIndex(df): 
#   # Create new column names
#   oldColumns = df.schema.names
#   newColumns = oldColumns + ["columnindex"]

#   # Add Column index
#   df_indexed = df.rdd.zipWithIndex().map(lambda (row, columnindex): \
#                                          row + (columnindex,)).toDF()

#   #Rename all the columns
#   new_df = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], 
#                   newColumns[idx]), xrange(len(oldColumns)), df_indexed)   
#   return new_df

# # Add index now...
# df1WithIndex = addColumnIndex(df1)
# df2WithIndex = addColumnIndex(df2)

# #Now time to join ...
# newone = df1WithIndex.join(df2WithIndex, col("columnindex"),
#                            'inner').drop("columnindex")

In [6]:
df_usv.where((df_usv._c20).isNull()).count()

0

In [ ]:
# df_svd = df_us.withColumn('_c0', df._c0)

In [7]:
df_c0 = df.select(df.columns[0])
df_c0.show()

+--------------------+
|                 _c0|
+--------------------+
|5c7727879dc6d68ae...|
|5c7727879dc6d68ae...|
|5c7727879dc6d68ae...|
|5c7727999dc6d68ae...|
|5c77279a9dc6d68ae...|
|5c77279a9dc6d68ae...|
|5c77279b9dc6d68ae...|
|5c77279e9dc6d68ae...|
|5c77279e9dc6d68ae...|
|5c77279f9dc6d68ae...|
|5c77279f9dc6d68ae...|
|5c7727a09dc6d68ae...|
|5c7727a19dc6d68ae...|
|5c7727a19dc6d68ae...|
|5c7727a39dc6d68ae...|
|5c7727a39dc6d68ae...|
|5c7727ae9dc6d68ae...|
|5c7727af9dc6d68ae...|
|5c7727b59dc6d68ae...|
|5c7727b79dc6d68ae...|
+--------------------+
only showing top 20 rows



In [ ]:
# df.na.drop(subset=["dt_mvmt"])

In [ ]:
#     // Array[Vector]转换成DataFrame
#     val df = sqlContext.createDataFrame(data.map(Tuple1.apply)).toDF("features")

#     // DataFrame转换成RDD
#     val df_To_rdd=df.select("features").map { case Row(v: Vector) => v}


In [ ]:
# from pyspark.sql.functions import col

# def addColumnIndex(df): 
#   # Create new column names
#   oldColumns = df.schema.names
#   newColumns = oldColumns + ["columnindex"]

#   # Add Column index
#   df_indexed = df.rdd.zipWithIndex().map(lambda row, columnindex: row + (columnindex,)).toDF()

#   #Rename all the columns
#   new_df = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], 
#                   newColumns[idx]), xrange(len(oldColumns)), df_indexed)   
#   return new_df

# # Add index now...
# df_idx = addColumnIndex(df)
# df_us_idx = addColumnIndex(df_us)

# #Now time to join ...
# df_svd = df_idx.join(df_us_idx, col("columnindex"),
#                            'inner').drop("columnindex")

In [ ]:
# firstDF = spark.range(3).toDF("myCol")
# newRow = spark.createDataFrame([[20]])
# firstDF.show()
# newRow.show()
# appended = firstDF.union(newRow)
# appended.show()
# display(appended)

In [9]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id

df_c0 = df_c0.withColumn("id1", monotonically_increasing_id())
# df_c0.printSchema()
df_usv = df_usv.withColumn("id2", monotonically_increasing_id())
# df_us.printSchema()
df_svd = df_c0.join(df_usv, df_c0.id1 == df_usv.id2, "inner")
# df_svd.count()
df_svd = df_svd.select([c for c in df_svd.columns if c not in {'id1','id2'}])
# df_svd.drop('id1')
# df_svd.printSchema()
df_svd.show()
df_svd.count()
df_svd.write.csv('res_comment_svd.csv')

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _c0|                 _c1|                _c10|                _c11|                _c12|                _c13|                _c14|                _c15|                _c16|                _c17|                _c18|                _c19|                 _c2|                _c20|                 _c3|                 _c4|                 _c5|                 _c6|                 _c7|                 _c8|                 _c9|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------